<a href="https://colab.research.google.com/github/AnilOsmanTur/Spatio-Temporal-Event-Prediction/blob/main/Prediction_with_dl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prediction wtih DL model

In [1]:
!pip install pytorch-lightning
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader
from torch.utils.data import random_split
import pytorch_lightning as pl
import numpy as np

pl.seed_everything(42) # for reproducablity
print('imports done')

     |████████████████████████████████| 808kB 19.1MB/s 
     |████████████████████████████████| 645kB 52.6MB/s 
     |████████████████████████████████| 112kB 59.1MB/s 
     |████████████████████████████████| 829kB 56.8MB/s 
     |████████████████████████████████| 276kB 48.8MB/s 
     |████████████████████████████████| 1.3MB 55.8MB/s 
     |████████████████████████████████| 296kB 55.1MB/s 
     |████████████████████████████████| 143kB 51.9MB/s 
  Created wheel for future: filename=future-0.18.2-cp37-none-any.whl size=491058 sha256=772f33398046f7a3c284f4452cee2f239692f54555628a2b2d80f52c21a329b2
  Stored in directory: /root/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
Successfully built future
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Found existing installation: future 0.16.0
    Uninstalling future-0.16.0:
      Successfully uninstalled future-0.16.0


Global seed set to 42


imports done


## Data Loading and Spliting

In [42]:
data = np.load('data.npy')
n_sample = data.shape[0]
flat_data = data.reshape(n_sample, -1)
print('data shape after flattening', flat_data.shape)
n_train = int(0.8 * n_sample )
n_test = n_sample - n_train
train_data = flat_data[:n_train]
test_data = flat_data[n_train:]
print('Training data split', train_data.shape)
print('Testing data split', test_data.shape)

data shape after flattening (4500, 9)
Training data split (3600, 9)
Testing data split (900, 9)


In [43]:
# data generator function to split and connect small sequences
def data_generator(data, seq_size=5):
  n_sample = data.shape[0]
  dataset = []
  labels = []
  for end in range(seq_size,n_sample):
    start = end - seq_size
    dataset.append(data[start:end].flatten())
    labels.append(data[end])
  return np.array(dataset).astype(np.float32), np.array(labels).astype(np.float32)
print('Done')

Done


In [44]:
train_set, train_labels = data_generator(train_data, seq_size=5)
test_set, test_labels = data_generator(test_data, seq_size=5)
print(train_set.shape, train_labels.shape)
print(test_set.shape, test_labels.shape)

(3595, 45) (3595, 9)
(895, 45) (895, 9)


## Model Creation

In [104]:
class SpatioTempPredictor(pl.LightningModule):
  def __init__(self, seen_size=5):
    super().__init__()
    self.in_size = seen_size * 9
    self.predictor = nn.Sequential(
      nn.Linear(self.in_size, 64),
      nn.ReLU(),
      nn.Linear(64, 16),

      nn.Linear(16, 64),
      nn.ReLU(),
      nn.Linear(64, 9),
      nn.Sigmoid()
      )
    self.euclidian_dist = nn.PairwiseDistance(p=2)

  def hamming_dist(self, a, b):
    return torch.count_nonzero(a!=b)

  def forward(self, x):
    prediction = self.predictor(x)
    return prediction

  def configure_optimizers(self):
    optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
    return optimizer

  def training_step(self, batch, batch_idx):
    x, y = batch
    x_hat = self.predictor(x)
    loss = F.mse_loss(x_hat, y)
    e_dist = self.euclidian_dist(x_hat, y).mean()
    h_dist = self.hamming_dist(x_hat, y).float()
    self.log('train_loss', loss)
    self.log('train_ed', e_dist)
    self.log('train_hd', h_dist)
    
    return loss

  def validation_step(self, batch, batch_idx):
    x, y = batch
    x_hat = self.predictor(x)
    loss = F.mse_loss(x_hat, y)
    e_dist = self.euclidian_dist(x_hat, y).mean()
    h_dist = self.hamming_dist(x_hat, y).float()
    self.log('val_loss', loss)
    self.log('val_ed', e_dist)
    self.log('val_hd', h_dist)
  
  def test_step(self, batch, batch_idx):
    # OPTIONAL
    x, y = batch
    x_hat = self.predictor(x)

    e_dist = self.euclidian_dist(x_hat, y)
    h_dist = self.hamming_dist(x_hat, y)
    loss = F.mse_loss(x_hat, y)
    output = {'test_loss': loss.cpu(), 'test_e_dist': e_dist.cpu(), 'test_h_dist': h_dist.cpu()}
    #print(output)
    return output


# data
dataset = list(zip(train_set, train_labels))
n_train = int(len(dataset)*0.9)
n_val = len(dataset) - n_train
train_split, val_split = torch.utils.data.random_split(dataset, [n_train, n_val])


train_loader = DataLoader(train_split, batch_size=16)
val_loader = DataLoader(val_split, batch_size=16)
test_loader = DataLoader(list(zip(test_set, test_labels)), batch_size=16)

# model
model = SpatioTempPredictor(seen_size=5)
print('Done')

Done


In [105]:
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
# training
early_stop_callback = EarlyStopping(
   monitor='val_loss',
   min_delta=0.00,
   patience=3,
   verbose=False,
   mode='max'
)
trainer = pl.Trainer(gpus=1, max_epochs=100, callbacks=[early_stop_callback])
trainer.fit(model, train_loader, val_loader)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name           | Type             | Params
----------------------------------------------------
0 | predictor      | Sequential       | 5.7 K 
1 | euclidian_dist | PairwiseDistance | 0     
----------------------------------------------------
5.7 K     Trainable params
0         Non-trainable params
5.7 K     Total params
0.023     Total estimated model params size (MB)


Global seed set to 42


In [85]:
# evaluation
trainer.test(model, test_loader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{}
--------------------------------------------------------------------------------


[{}]